In [2]:
import cvxpy as cp
import numpy as np
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import networkx as nx
import random


from typing import Tuple
from itertools import product

## Questão 1: Arranha-céus!

### a)

### b)

### c)

\begin{align*}
    \text{Encontrar }   & x_{ijk} \hspace{20pt} \text{(Alocar o número } k \text{ à posição } (i,j) \text{ do tabuleiro)}\\
    \text{sujeito a }   & \sum_{k} x_{ijk} = 1 \quad \forall  i,j  \hspace{20pt} \text{(Alocar apenas um número em cada casa)} \\
                        & \sum_{i} x_{ijk} = 1 \quad \forall  j,k  \hspace{20pt} \text{(Apenas um número em cada coluna)} \\
                        & \sum_{j} x_{ijk} = 1 \quad \forall  i,k  \hspace{20pt} \text{(Apenas um número em cada linha)} \\
                        & 0 \leq \sum_k( x_{ik} - x_{lk} ) \leq ny_{im} \quad \forall i \in r_m; \: \forall l \in 1,...,i-1; \: \forall r_m \in R \\
                        & x_{ijk} = 1 \quad \forall i,j,k: \: \text{board}(i,j) = k \hspace{20pt} \text{(Fixar valores do tabuleiro inicial)}\\
                        & x_{ijk}\in \mathbb{B} \quad \forall  i,j,k \in 0,...,n  \hspace{20pt} \text{(Variáveis de decisão binárias - alturas dos prédios)} \\
                        & y_{lm} \in \mathbb{B} \quad \forall  l\in 0,...,n; \: \forall m \in R \hspace{20pt} \text{(Variáveis auxiliares - indicatrizes de visibilidade)} \\
                        & \\
    \text{onde }        & R \text{ é o conjunto de fileiras onde há restrição de visão, e } r_i \text{ é o conjunto de casas da i-ésima fileira, ordenadas à partir de seu início.}
\end{align*}

No modelo acima, é importante ressaltar que os índices $l$ das variáveis de visibilidade indexam as casas do tabuleiro em uma dada fileira, _na ordem definida pela fileira_. Ou seja, os índices $l$ podem  indexar as casas na ordem inversa da indexação do tabuleiro, se a restrição de visibilidade for dada à partir da direita ou do lado de baixo do tabuleiro.

In [ ]:
def SolveSkyscraper(board:np.ndarray, sightlines:np.ndarray) -> np.ndarray:

    boardSize = len(board[:,0])
    digits = range(boardSize)
    nbSightlines = len(sightlines)

    # decision variables
    decisionVars = np.empty((boardSize, boardSize, boardSize), dtype=object)
    for i in digits:
        for j in digits:
            for k in digits:
                decisionVars[i,j,k] = cp.Variable(boolean=True, name=f"x_{i}{j}{k}")
    
    sightlineCounters = np.empty((nbSightlines, boardSize), dtype=object)
    for l in digits:
        for m in nbSightlines:
            sightlineCounters[l,m] = cp.Variable(boolean=True, name=f"y_{l}{m}")

    # constraints
    constraints = []
    # one single value per cell
    for i in digits:
        for j in digits:
            constraints += [np.sum(decisionVars[i,j,:]) == 1]

    # uniqueness in every column
    for j in digits:
        for k in digits:
            constraints += [np.sum(decisionVars[:,j,k]) == 1]
    
    # uniqueness in every row
    for i in digits:
        for k in digits:
            constraints += [np.sum(decisionVars[i,:,k]) == 1]

    # initial board values 
    for i in digits:
        for j in digits:
            boardValue = board[i,j]
            if boardValue != 0:
                constraints.append(decisionVars[i, j, board[i,j] - 1] == 1)

    objective = cp.Minimize(0)

    lp = cp.Problem(objective, constraints)
    lp.solve(solver="MOSEK", verbose=False)

    return decisionVars

## Questão 2: Plantação de tomates!

### a) Estratégia gulosa
A estratégia gulosa funciona para o exemplo simples onde o jardim é dado por

| 5 | 3 | 5 |

Um contra exemplo à essa estratégia é o jardim 

| 3 | 5 | 3 |

Nesse último caso, a estratégia gulosa nos dá uma delicioncisade total de 5, sendo que o ótimo é na realidade 6.

### b)

Para um problema de tamanho $n$, podemos escrever a sua solução ótima em termos de soluções ótimas de problemas menores. Preenchendo o jardim da esquerda para a direita (onde indexamos a primeira casa por $0$ e a última por $n$), temos que a solução de tamanho $S_n$ pode ser escrita como 

$$ S_n = \max\{S_{n-1},\, S_{n-2} + T_n\} $$

Isso pode ser interpretado como o máximo dentre duas opções possíveis: 

1) Não plantar um tomate na última casa, pois a penúltima já contém um tomate
2) Plantar um tomate na última casa, ao custo de não ter um tomate plantado na penúltima casa, para manter a solução factível

Teremos também dois casos base para a recursão, sendo eles $S_1 = T_1$ e $S_2 = \max\{ T_1,\, T_2 \}$.

Esse no entanto não é o método mais eficiente de se implementar a resolução do problema. Podemos usar as técnicas de programação dinâmica vistas em aula para calcular as soluções de cada subproblema de forma iterativa, evitando o recálculo de soluções. Como as soluções são indexadas por apenas uma variável, podemos criar um algoritmo de complexidade O(n) para resolver esse problema.

### c) Programa linear

\begin{align*}
    \text{Maximizar }   & \sum_{i=1}^{n} T_i x_i \hspace{20pt} \text{(Deliciosidade total)}\\
    \text{sujeito a }   & \; x_i + x_{i+1} \leq 1 \quad \forall i \in \{1,..., n-1\}  \hspace{20pt} \text{(Não plantar dois tomates em casas adjacentes)} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i  \hspace{20pt} \text{(Variáveis de decisão binárias - plantar ou não plantar tomate em cada casa)} \\
                        & T_{i}\in \mathbb{Z} \quad \forall  i  \hspace{20pt} \text{(Deliciosidades inteiras e positivas)} \\
\end{align*}

### d)



In [66]:
def TomatoLinearProgram(deliciousnessMap:np.ndarray):
    
    probSize = len(deliciousnessMap)

    # decision variables
    decisionVars = cp.Variable(probSize, boolean=True, name="stop")

    # constraints
    constraints = []
    # no adjacent tomatoes
    for i in range(probSize-1):
        constraints += [decisionVars[i] + decisionVars[i+1] <= 1]

    objective = cp.Maximize(decisionVars @ deliciousnessMap)

    # solving 
    lp = cp.Problem(objective, constraints)
    lp.solve(solver="MOSEK", verbose=False)

    # printing solution
    print("Solution: ", decisionVars.value)
    print("Objective: ", lp.value)
    
    return 


def TomatoRecursion(deliciousnessMap:np.ndarray) -> np.ndarray:

    if len(deliciousnessMap) == 1:
        return deliciousnessMap[0]
    elif len(deliciousnessMap) == 2:
        return max(deliciousnessMap[0], deliciousnessMap[1])
    else:
        return max(TomatoRecursion(deliciousnessMap[:-1]), TomatoRecursion(deliciousnessMap[:-2]) + deliciousnessMap[-1])


def TomatoIteration(deliciousnessMap:np.ndarray) -> np.ndarray:
    
    probSize = len(deliciousnessMap)
    solutions = np.zeros(probSize)

    # base cases
    solutions[0] = deliciousnessMap[0]
    solutions[1] = max(deliciousnessMap[0], deliciousnessMap[1])
    
    # building new solutions based on previous known solutions
    for i in range(2, probSize):
        solutions[i] = max(solutions[i-1], solutions[i-2] + deliciousnessMap[i])
        
    return solutions


deliciousnessMap1 = [5, 12, 10, 7, 15, 10, 11, 5, 8, 10]
deliciousnessMap2 = [10, 12, 5, 12, 20, 18, 5, 3, 2, 8]

testCase = deliciousnessMap1

print("Recursive solution: ", TomatoRecursion(testCase))
print()
print("Iterative solution: ", TomatoIteration(testCase)[-1])
print()
print("Linear programming solution: ")
TomatoLinearProgram(testCase)

Recursive solution:  51

Iterative solution:  51.0

Linear programming solution: 
Solution:  [1. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
Objective:  51.0


## Questão 3: Dois problemas fundamentais em grafos bipartidos

### a)

Dado um grafo $G(V, E)$, observamos primeiramente que um emparelhamento qualquer $Y$ pode envolver, no melhor dos casos, todos os seus nós. Isso implica que  $|Y| <= |V|/2$ já que cada aresta de $Y$ não pode compartilhar nós com outras, e cada aresta cobre exatamente 2 nós. Já a cobertura 

### b) Emparelhamento máximo

\begin{align*}
    \text{Maximizar }   & \sum_{(i,j)\in E} x_{ij} \hspace{20pt} \text{(Maximizar conjunto de emparalhamento)}\\
    \text{sujeito a }   & \sum_{j} x_{ij} = 1 \quad \forall i \in V  \hspace{20pt} \text{(Apenas uma aresta chegando em um vértice }i\text{)} \\
                        & \sum_{i} x_{ij} = 1 \quad \forall j \in V  \hspace{20pt} \text{(Apenas uma aresta saindo de um vértice }j\text{)} \\
                        & x_{ij}\in \mathbb{B} \quad \forall  (i,j)\in E  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
                                                & \\
    \text{onde }        & G(V,E) \text{ é um grafo com conjunto de nós } V \text{e conjunto de arestas } E     
\end{align*}

### c) Cobertura mínima

\begin{align*}
    \text{Minimizar }   & \sum_{i\in V} x_{i} \hspace{20pt} \text{(Minimizar conjunto de cobertura)}\\
    \text{sujeito a }   & \; x_{i} + x_{j} >= 1 \quad \forall (i,j) \in E  \hspace{20pt} \text{(Toda aresta possui um de seus nós na cobertura} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i\in V  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
                        & \\
    \text{onde }        & G(V,E) \text{ é um grafo com conjunto de nós } V \text{e conjunto de arestas } E             
\end{align*}

### d)



## Questão 4: O Princípio da Casa dos Pombos

### a)

### b)

### c)



## Questão 5: Pudim, o pinguim comilão.

### a)

O problema só é factível se não houver nenhum segmento da viagem de comprimento maior de que $m$, a distância máxima que Pudim pode viajar sem comer. Mais formalmente, o problema é factível somente se $F_i - F_{i-1} \leq m \quad \forall i \in \{ 2,...,n \}$.

### b)

\begin{align*}
    \text{Minimizar }   & \sum_{i=1}^{n} x_i \hspace{20pt} \text{(Total de paradas realizadas)}\\
    \text{sujeito a }   & \sum_{i\in Cj}x_i \geq  1 \quad \forall j \in \{1,..., n\}  \hspace{20pt} \text{(Toda parada possui uma parada anterior dentro do alcance máximo)} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i  \hspace{20pt} \text{(Variáveis de decisão binárias - parar ou não no buraco de pesca)} \\
                        & F_{i}\in \mathbb{Z} \quad \forall  i  \hspace{20pt} \text{(Distâncias inteiras e positivas)} \\
                        & \\
    \text{onde }   & Cj \equiv \{ i:\, F_j - F_i <= m \} \hspace{20pt} \text{(Conjunto de paradas anteriores a } j \text{ a uma distância menor que } m\text{)}
\end{align*}

É importante ressaltar que, na modelagem acima, uma instância infactível do problema em teoria não gerará um modelo linear infactível. A infactibilidade do modelo no solver se dá devido à adição de uma restrição onde a soma de um conjunto vazio de variáveis de decisão deve ser maior ou igual a $1$ (em uma instância infactível, pelo menos um dos conjuntos $C_j$ é vazio). No entanto, isso poderia também ser interpretado matematicamente como uma restrição vazia, então algum cuidado deve ser tomado na interepretação dos resultados em função da implementação usada.

### c)

Seja $S(i)$ a solução ótima para o percurso até a parada $i$, sendo então o conjunto de paradas restantes do trajeto $R = \{ i+1,..,n \}$. Provaremos por absurdo que a escolha gulosa é a melhor escolha para o próximo passo da indução. Se a escolha gulosa é dada pela parada $k\in R$, mas ela não minimiza $|S(k)|$, então temos duas alternativas:

1) Existe algum $k'>k$ ao alcance de Pudim quando este está em $i$. Nesse caso, temos um absurdo pois a escolha gulosa implica em selecionar a última parada factível à partir de $i$.
2) A suposta solução melhor que a gulosa envolve algum $k' < k$. Como $k$ está ao alcance de Pudim em $i$, então podemos pular $k'$ e continuar com o mesmo número de paradas. 

Logo, se $S(i)$ é a solução ótima para $\{1,...,i\}$, então $S(k)$ dada pela estratégia gulosa é a solução ótima para $\{1,...,i,...,k\}$. O caso base da indução é dado por $S(1) = \{0\}$, o ponto de início da viagem.  

[talvez seja necessário usar indução completa sobre todas as paradas ao alcance de pudim em $i$, ao invés do raciocínio acima] 

In [69]:
# problem input
F = [0, 3, 4, 6, 10, 16, 18]
m = 4

# decision variables
decisionVars = cp.Variable(len(F), boolean=True, name="stop")

# constraints
constraints = []

# creating list of valid previous stops for each stop in the journey
allValidStops = []
for i in range(1, len(F)):
    validStops = []
    for j in range(i-1, -1, -1):
        if F[i] - F[j] <= m:
            validStops += [j]
    allValidStops.append(validStops)

# all stops must have at least one previous valid stop
for i in range(len(F)-1):
    constraints += [cp.sum(decisionVars[allValidStops[i]]) >= 1]

# start and end of the journey
constraints += [decisionVars[-1] == 1]
constraints += [decisionVars[0] == 1]

objective = cp.Minimize(cp.sum(decisionVars))

# solving 
lp = cp.Problem(objective, constraints)
lp.solve(solver="MOSEK", verbose=False)

# printing model and solution
print(lp)
print()
print("Solução: ", decisionVars.value)

minimize Sum(stop, None, False)
subject to 1.0 <= Sum(stop[0], None, False)
           1.0 <= Sum(stop[1, 0], None, False)
           1.0 <= Sum(stop[2, 1], None, False)
           1.0 <= Sum(stop[3], None, False)
           1.0 <= Sum(stop[], None, False)
           1.0 <= Sum(stop[5], None, False)
           stop[6] == 1.0
           stop[0] == 1.0

Solução:  None
